In [ ]:
import numpy as np
import pandas as pd
import csv
from collections import Counter  

In [ ]:
all_bots = pd.read_csv('botnames.csv')
all_bot_names = set()
for i in all_bots.itertuples():
    all_bot_names.add(i[2])

In [ ]:
Data = pd.read_csv('shares_df_2016.csv')
Data = Data.drop_duplicates()

all_bots_in_2018_dataset = set()

In [ ]:
all_non_suspects_after_threshold_one = set()
all_suspects_after_threshold_one = set()
unique_accounts= set()

In [ ]:
Data_for_second_threshold = []
for datarow in Data.itertuples():
    coordinated = datarow[6]
    Screen_name_from = datarow[3]
    unique_accounts.add(Screen_name_from)

    # for finding all the bots that are present in the 2016 dataset
    if Screen_name_from in all_bot_names:
        all_bots_in_2018_dataset.add(Screen_name_from)

    if coordinated:
        tid = datarow[1]
        retweet_tid = datarow[2]
        Screen_name_to = datarow[4]
        Data_for_second_threshold.append([tid, retweet_tid, Screen_name_from, Screen_name_to])

        all_suspects_after_threshold_one.add(Screen_name_from)

    else:
        all_non_suspects_after_threshold_one.add(Screen_name_from)

In [ ]:
#tier2 calculation
Data_for_second_threshold = []
for datarow in Data.itertuples():
    coordinated = datarow[6]
    Screen_name_from = datarow[3]
    unique_accounts.add(Screen_name_from)

    # for finding all the bots that are present in the 2016 dataset
    if Screen_name_from in all_bot_names:
        all_bots_in_2018_dataset.add(Screen_name_from)


    tid = datarow[1]
    retweet_tid = datarow[2]
    Screen_name_to = datarow[4]
    Data_for_second_threshold.append([tid, retweet_tid, Screen_name_from, Screen_name_to])

    all_suspects_after_threshold_one.add(Screen_name_from)

#     else:
#         all_non_suspects_after_threshold_one.add(Screen_name_from)

In [ ]:
Dict = {}
for i in Data_for_second_threshold:
    tid = i[0]
    retweet_tid = i[1]
    Screen_name_from = i[2]
    Screen_name_to = i[3]

    if retweet_tid not in Dict:
        Dict[retweet_tid] = set()
    Dict[retweet_tid].add(Screen_name_from)

In [ ]:
print(len(Dict))

In [ ]:
Final_result_Dict = {}

for ind,i in enumerate(Dict):
    Set_for_this_retweet =list(Dict[i])
    set_length = len(Set_for_this_retweet)

    if set_length > 1:
        for i in range(set_length - 1):
            for j in range(i + 1, set_length):
                Screen_name_from_one = Set_for_this_retweet[i]
                Screen_name_from_two = Set_for_this_retweet[j]
                list_names = [Screen_name_from_one,Screen_name_from_two]
                list_names.sort()
                if tuple(list_names) in Final_result_Dict:
                    Final_result_Dict[tuple(list_names)] = Final_result_Dict[tuple(list_names)] + 1
                
                else:
                    Final_result_Dict[tuple(list_names)] = 1

In [ ]:
final_bots = set()
for i in Final_result_Dict:
    if Final_result_Dict[i] > 12:
        final_bots.add(i[0])
        final_bots.add(i[1])

final_bots = list(final_bots)

count_actual_bots = []
for i in final_bots:
    if i in all_bots_in_2018_dataset:
        count_actual_bots.append(i)

print(len(count_actual_bots))
print(len(final_bots))
print(len(all_bots_in_2018_dataset))

In [ ]:
tier1_final_bots = list(final_bots)
with open("bots_q_30_2016_tier1.txt" , 'w' ) as filehandle:
    for listitem in tier1_final_bots:
        filehandle.write('%s\n' % listitem)

In [ ]:
# tier2_final_bots = list(final_bots)
with open('bots_q_20_2016_tier1.txt', 'r') as in_file:
    tier1_final_bots = in_file.read().split('\n')


In [ ]:
print(len(tier1_final_bots))

In [ ]:
tier2_final_bots = list(set(final_bots))
print(len(tier2_final_bots))

In [ ]:
total_bots_from_tiers = list(set(tier1_final_bots+tier2_final_bots))
count_actual_bots_both_tiers = []
for i in total_bots_from_tiers:
    if i in all_bots_in_2018_dataset:
        count_actual_bots_both_tiers.append(i)

recall = len(count_actual_bots_both_tiers) / len(all_bots_in_2018_dataset)


precision = len(count_actual_bots_both_tiers)/len(total_bots_from_tiers)


In [ ]:
Total_predictions = len(unique_accounts)

TP = len(count_actual_bots_both_tiers)
FN = len(all_bots_in_2018_dataset) - len(count_actual_bots_both_tiers)
FP = len(total_bots_from_tiers) - len(count_actual_bots_both_tiers)
TN = Total_predictions - TP- FN- FP

correct_predictions = TP + TN
wrong_predictions = FN + FP
Accuracy = correct_predictions/Total_predictions
print("correctly  bots {}".format(len(set(count_actual_bots_both_tiers))))
print("Total bots predicted by Algo: {}".format(len(total_bots_from_tiers)))
print("Recall : {}".format(recall))
print("Precision:{}".format(precision))
print("Accuracy:{}".format(Accuracy))

In [ ]:
with open("Ds-1_2016_predicted_bots_correctly.txt" , 'w' ) as filehandle:
    for listitem in count_actual_bots_both_tiers:
        filehandle.write('%s\n' % listitem)

In [ ]:
with open("correct_bots_tier1_20_tier2_12_2016.txt" , 'w' ) as filehandle:
    for listitem in count_actual_bots_both_tiers:
        filehandle.write('%s\n' % listitem)